# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096798


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:33,  3.35s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:04,  2.40s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.76s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.30s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:11<00:01,  3.85it/s]

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  3.95it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  4.05it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.13it/s]

nurenai                               0.011011
VoyagerST                             0.000757
Quarkgc                               0.000129
jnarayanbvg                           0.007141
Jonas_Cross                           0.000435
not-logged-in-5ba9a08878d4b5e44953    0.203825
not-logged-in-c6917d9b19f41e6c2995    0.002266
awright5                              0.002852
Haransis                              0.016378
not-logged-in-74261f68b83c7ae82ce3    0.000255
gracie_solveig                        0.000703
not-logged-in-5f30b2bb9b7f9d49da41    0.000579
AdamMorse                             0.002760
Theaker                               0.002913
mariafarhat                           0.002913
BraxtonKinney                         0.000212
acapirala                             0.000437
haydenmanninen                        0.000226
alexmnelson18                         0.000224
Lavadude                              0.048316
cowslip                               0.020699
Martin97     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())